In [120]:
import numpy

class matrix_factorization_learning_model():
    
    def __init__(self,R,a,b,it,k):
        self.R=R
        self.user_n, self.item_n=R.shape
        self.alpha=a
        self.beta=b
        self.iterations=it
        self.K=k
        
    def learn(self):
        self.b_users=numpy.zeros(self.user_n)
        self.b_items=numpy.zeros(self.item_n)
        self.b=numpy.mean(self.R[numpy.where(self.R!=0)])
        
        self.P=numpy.random.normal(scale=1./self.K,size=(self.user_n,self.K))
        self.Q=numpy.random.normal(scale=1./self.K,size=(self.item_n,self.K))
        rng=numpy.random.default_rng()
        
        self.values=numpy.empty((self.R.size,3),numpy.int8)
        k=0
        for i in range (self.user_n):
            for j in range (self.item_n):
                if ((self.R[i][j])!=0):
                    self.values[k]=[i,j,self.R[i][j]]
                k+=1
                
        samplesize=5
        self.samples=rng.choice(self.values,samplesize,False)
        
        
        self.notsampled=numpy.empty((0,3),numpy.int8)
        for row in self.values:
            if(not (any(numpy.array_equal(x,row) for x in self.samples))):
                self.notsampled=numpy.append(self.notsampled,[row],0)
        
        
        print(self.samples)
        print(self.notsampled)
        print(self.values)
        for i in range (self.iterations):
            self.sgd()
            ls=self.leastCertain()
            
            
        
            
    def leastCertain(self):
        Q=numpy.absolute(self.complete())
        return numpy.unravel_index(numpy.argmin(Q),Q.shape)

    def sgd(self):
        for i, j, r in self.samples:
            prediction = self.b_users[i]+self.b_items[j]+self.b+self.P[i,:].dot(self.Q[j,:].T)
            e = (r - prediction)
            
            self.b_users[i]+=self.alpha*(e-self.beta*self.b_users[i])
            self.b_items[j]+=self.alpha*(e-self.beta*self.b_items[j])
            
            self.P[i,:]+=self.alpha*(e*self.Q[j,:]-self.beta*self.P[i,:])
            self.Q[j,:]+=self.alpha*(e*self.P[i,:]-self.beta*self.Q[j,:])
            
    def get_rating(self,i,j):
        return self.b_users[i]+self.b_items[j]+self.b+self.P[i,:].dot(self.Q[j,:].T)
    
    def complete(self):
        return self.P.dot(self.Q.T)+self.b+self.b_users[:,numpy.newaxis]+self.b_items[numpy.newaxis,:]
    
def voting(C):
    D=numpy.zeros(shape=C[0].R.shape)
    for i in range (D.shape[0]):
        for j in range (D.shape[1]):
            vn=0
            vp=0
            for item in C:
                if (item.complete()[i][j]>0):
                    vp+=1
                else:
                    vn+=1
            if (vp==0 or vn==0):
                s1=0
                s2=0
            else:
                s1=vp/len(C)*numpy.log(vp/len(C))
                s2=vn/len(C)*numpy.log(vn/len(C))
            D[i][j]=-s1-s2

    return numpy.unravel_index(numpy.argmax(D),D.shape)
    
def twoWayVoting(C):
    c1=C[0].complete()
    c2=C[1].complete()
    D=numpy.absolute(numpy.subtract(c1,c2))
    print(numpy.unravel_index(numpy.argmax(D),D.shape))
    return numpy.amax(D)
    
        
        

In [121]:
M = numpy.array([
    [1, -1, 1, 1],
    [1, 1, -1, 1],
    [-1, -1, 1, 1],
    [1, 1, 1, -1],
    [1, 1, -1, -1],
])

model = matrix_factorization_learning_model(M, 0.1, 0.01, 5, 2)
model.learn()


[[ 0  0  1]
 [ 2  2  1]
 [ 0  3  1]
 [ 4  0  1]
 [ 3  3 -1]]
[[ 0  1 -1]
 [ 0  2  1]
 [ 1  0  1]
 [ 1  1  1]
 [ 1  2 -1]
 [ 1  3  1]
 [ 2  0 -1]
 [ 2  1 -1]
 [ 2  3  1]
 [ 3  0  1]
 [ 3  1  1]
 [ 3  2  1]
 [ 4  1  1]
 [ 4  2 -1]
 [ 4  3 -1]]
[[ 0  0  1]
 [ 0  1 -1]
 [ 0  2  1]
 [ 0  3  1]
 [ 1  0  1]
 [ 1  1  1]
 [ 1  2 -1]
 [ 1  3  1]
 [ 2  0 -1]
 [ 2  1 -1]
 [ 2  2  1]
 [ 2  3  1]
 [ 3  0  1]
 [ 3  1  1]
 [ 3  2  1]
 [ 3  3 -1]
 [ 4  0  1]
 [ 4  1  1]
 [ 4  2 -1]
 [ 4  3 -1]]


In [75]:
M = numpy.array([
    [1, -1, 1, 1],
    [1, 1, -1, 1],
    [-1, -1, 1, 1],
    [1, 1, 1, -1],
    [1, 1, -1, -1],
])
print(M)
print()
C=[]
print("Least Certain Instance's Indexes by models: ")
for i in range(5):
    model = matrix_factorization_learning_model(M, 0.1, 0.01, 25, 2)
    model.learn()
    print(model.leastCertain())
    C.append(model)
for i in range(5):
    model = matrix_factorization_learning_model(M, 0.1, 0.01, 15, 2)
    model.learn()
    print(model.leastCertain())
    C.append(model)

print("\nIndex of the least agreed instance by committee:")
d=voting(C)
print(d)
C=numpy.reshape(C,(2,5))
print("\nBiggest differnce in two models:")
twoWayVoting(C.reshape(10,order='F'))





[[ 1 -1  1  1]
 [ 1  1 -1  1]
 [-1 -1  1  1]
 [ 1  1  1 -1]
 [ 1  1 -1 -1]]

Least Certain Instance's Indexes by models: 
w
w
w
w
w
w
w
w
w
w
w
w
w
w
w
[]


IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices